In [6]:
img = "dataset/01020.jpg"

In [7]:
import cv2
import numpy as np

# Cargar los archivos YOLO
weights_path = 'yolov3.weights'
config_path = 'yolov3.cfg'
names_path = 'coco.names'

# Cargar las etiquetas de las clases
with open(names_path, 'r') as f:
    classes = [line.strip() for line in f.readlines()]

# Cargar la red YOLO
net = cv2.dnn.readNet(weights_path, config_path)

# Cargar la imagen
image = cv2.imread(img)

img_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

img_hsv[:, :, 2] = cv2.equalizeHist(img_hsv[:, :, 2])

image = cv2.cvtColor(img_hsv, cv2.COLOR_HSV2BGR)

height, width = image.shape[:2]

# Crear el blob de entrada
blob = cv2.dnn.blobFromImage(image, 1/255.0, (416, 416), swapRB=True, crop=False)
net.setInput(blob)

# Obtener las salidas de las capas YOLO correctamente
layer_names = net.getLayerNames()

# Si la función getUnconnectedOutLayers devuelve un número en lugar de una lista, lo manejamos correctamente.
try:
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
except TypeError:
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

# Forward pass para obtener las detecciones
outputs = net.forward(output_layers)

# Procesar las detecciones
class_ids = []
confidences = []
boxes = []

coordinates_list = []

for output in outputs:
    for detection in output:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        
        if confidence > 0.5:
            # Convertir las coordenadas
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)

            coordinates = (center_x, center_y, w, h)

            # Calcular el box
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)

            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)
            
            coordinates_list.append(coordinates)

# Aplicar non-maxima supresión para eliminar duplicados
indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
print(indices)

coordinates_list_v2 = [[coordinates_list[i], str(classes[class_ids[i]]), confidences[i]] for i in indices]

# Dibujar los bounding boxes en la imagen
for i in indices.flatten():
    x, y, w, h = boxes[i]
    label = str(classes[class_ids[i]])
    confidence = confidences[i]
    
    # Dibujar el rectángulo y la etiqueta
    color = (0, 255, 0)
    cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
    cv2.putText(image, f'{label} {confidence:.2f}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

# Mostrar la imagen
cv2.imshow('YOLO Object Detection', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


[3 7 6 0 8]


In [8]:
coordinates_list_v2

[[(544, 349, 207, 163), 'car', 0.9618672132492065],
 [(408, 186, 46, 56), 'traffic light', 0.7441791296005249],
 [(657, 118, 47, 67), 'traffic light', 0.595501184463501],
 [(1126, 252, 306, 428), 'truck', 0.5650248527526855],
 [(377, 311, 57, 40), 'car', 0.5628335475921631]]

In [9]:
import pandas as pd

dataset = pd.read_excel('dataset_database.xlsx')
dataset

C:\Users\mario\AppData\Local\Temp\ipykernel_18936\830637486.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


,subject,collision
0,00001.jpg,n
1,00002.jpg,n
2,00003.jpg,n
3,00004.jpg,n
4,00005.jpg,n
...,...,...
9995,09996.jpg,n
9996,09997.jpg,n
9997,09998.jpg,n
9998,09999.jpg,y


In [10]:
def get_image_info(dataset, img):

    image = cv2.imread("dataset/" + img)
    img_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    img_hsv[:, :, 2] = cv2.equalizeHist(img_hsv[:, :, 2])
    image = cv2.cvtColor(img_hsv, cv2.COLOR_HSV2BGR)

    height, width = image.shape[:2]

# Crear el blob de entrada
    blob = cv2.dnn.blobFromImage(image, 1/255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)

# Obtener las salidas de las capas YOLO correctamente
    layer_names = net.getLayerNames()

# Si la función getUnconnectedOutLayers devuelve un número en lugar de una lista, lo manejamos correctamente.
    try:
        output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
    except TypeError:
        output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

# Forward pass para obtener las detecciones
    outputs = net.forward(output_layers)

# Procesar las detecciones
    class_ids = []
    confidences = []
    boxes = []

    coordinates_list = []

    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
        
            if confidence > 0.5:
                # Convertir las coordenadas
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                coordinates = (center_x, center_y, w, h)

                # Calcular el box
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
            
                coordinates_list.append(coordinates)

    # Aplicar non-maxima supresión para eliminar duplicados
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    coordinates_list_v2 = [[coordinates_list[i], str(classes[class_ids[i]]), confidences[i]] for i in indices]
    return coordinates_list_v2

In [15]:
dataset['info'] = np.nan

In [16]:
dataset

,subject,collision,info
0,00001.jpg,n,NaN
1,00002.jpg,n,NaN
2,00003.jpg,n,NaN
3,00004.jpg,n,NaN
4,00005.jpg,n,NaN
...,...,...,...
9995,09996.jpg,n,NaN
9996,09997.jpg,n,NaN
9997,09998.jpg,n,NaN
9998,09999.jpg,y,NaN


In [18]:
dataset.to_excel('dataset_database_1.xlsx', index=False)

In [19]:
dataset

,subject,collision,info
0,00001.jpg,n,NaN
1,00002.jpg,n,NaN
2,00003.jpg,n,NaN
3,00004.jpg,n,NaN
4,00005.jpg,n,NaN
...,...,...,...
9995,09996.jpg,n,NaN
9996,09997.jpg,n,NaN
9997,09998.jpg,n,NaN
9998,09999.jpg,y,NaN
